In [1]:
import pandas as pd 

In [2]:
df=pd.read_pickle('../data/preprocessed_data.pkl')

In [3]:
df

,time_limit,memory_limit,title,description,input_format,output_format,rating,tags,difficulty,combined_text,tokens,tokens_clean,algo_count,tokens_lemma,word_count,token_count,complexity_terms,tfidf_text
0,1.0,256.0,Minimal Coprime,"Today, Little John used all his savings to buy...",Each test contains multiple test cases. The fi...,"For each test case, output the number of minim...",800.0,['math' 'number theory'],Easy,Minimal Coprime Today Little John used all hi...,"[Minimal, Coprime, Today, Little, John, used, ...","[Minimal, Coprime, Today, Little, John, used, ...",1,"[Minimal, Coprime, Today, Little, John, used, ...",160,116,0,Minimal Coprime Today Little John used saving ...
1,1.5,256.0,Subsequence Update,After Little John borrowed expansion screws fr...,Each test contains multiple test cases. The fi...,"For each test case, output the minimum value o...",1100.0,['constructive algorithms' 'data structures' '...,Easy,Subsequence Update After Little John borrowed ...,"[Subsequence, Update, After, Little, John, bor...","[Subsequence, Update, After, Little, John, bor...",1,"[Subsequence, Update, After, Little, John, bor...",308,205,1,Subsequence Update After Little John borrowed ...
2,2.0,256.0,Remove Exactly Two,"Recently, Little John got a tree from his aunt...",Each test contains multiple test cases. The fi...,"For each test case, output the maximum number ...",1600.0,['brute force' 'data structures' 'dfs and simi...,Medium,Remove Exactly Two Recently Little John got a...,"[Remove, Exactly, Two, Recently, Little, John,...","[Remove, Exactly, Two, Recently, Little, John,...",2,"[Remove, Exactly, Two, Recently, Little, John,...",268,180,1,Remove Exactly Two Recently Little John got tr...
3,2.0,256.0,Game With Triangles,Even Little John needs money to buy a house. B...,Each test contains multiple test cases. The fi...,"For each test case, given that the maximum num...",2000.0,['binary search' 'brute force' 'data structure...,Hard,Game With Triangles Even Little John needs mon...,"[Game, With, Triangles, Even, Little, John, ne...","[Game, With, Triangles, Even, Little, John, ne...",0,"[Game, With, Triangles, Even, Little, John, ne...",443,299,2,Game With Triangles Even Little John need mone...
4,2.0,512.0,Triangle Tree,"One day, a giant tree grew in the countryside....",Each test contains multiple test cases. The fi...,"For each test case, output the answer on a sep...",2300.0,['data structures' 'dfs and similar' 'dp' 'gre...,Hard,Triangle Tree One day a giant tree grew in th...,"[Triangle, Tree, One, day, a, giant, tree, gre...","[Triangle, Tree, One, day, giant, tree, grew, ...",2,"[Triangle, Tree, One, day, giant, tree, grew, ...",334,235,0,Triangle Tree One day giant tree grew countrys...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10019,1.0,256.0,Hossam and (sub-)palindromic tree,Hossam has an unweighted tree $$$G$$$ with let...,The first line contains one integer $$$t$$$ ($...,For each test case output one integer — the le...,2100.0,['brute force' 'data structures' 'dfs and simi...,Hard,Hossam and sub palindromic tree Hossam has a...,"[Hossam, and, sub, palindromic, tree, Hossam, ...","[Hossam, sub, palindromic, tree, Hossam, unwei...",2,"[Hossam, sub, palindromic, tree, Hossam, unwei...",412,248,1,Hossam sub palindromic tree Hossam unweighted ...
10020,2.0,256.0,Hossam and a Letter,Hossam bought a new piece of ground with lengt...,The first line of the input contains two integ...,Print a single integer — the maximum number of...,2500.0,['brute force' 'dp' 'implementation' 'two poin...,Hard,Hossam and a Letter Hossam bought a new piece ...,"[Hossam, and, a, Letter, Hossam, bought, a, ne...","[Hossam, Letter, Hossam, bought, new, piece, g...",0,"[Hossam, Letter, Hossam, bought, new, piece, g...",328,203,2,Hossam Letter Hossam bought new piece ground l...
10021,1.5,256.0,Hossam and Range Minimum Query,Hossam gives you a sequence of integers $$$a_1...,The first line of the input contains one int

In [4]:
y_class=df['difficulty']
y_score=df['rating']

Applying tfidf to convert text data into numbers.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(
    token_pattern=r"(?u)\b\w+\b",
    ngram_range=(1, 4),
    max_features=3000, 
    stop_words='english', 
    sublinear_tf=True
)


In [6]:
df['tfidf_text']

0        Minimal Coprime Today Little John used saving ...
1        Subsequence Update After Little John borrowed ...
2        Remove Exactly Two Recently Little John got tr...
3        Game With Triangles Even Little John need mone...
4        Triangle Tree One day giant tree grew countrys...
                               ...                        
10019    Hossam sub palindromic tree Hossam unweighted ...
10020    Hossam Letter Hossam bought new piece ground l...
10021    Hossam Range Minimum Query Hossam give sequenc...
10022    Split Kawashiro Nitori girl love competitive p...
10023    Max Mex You given multiset S initially consist...
Name: tfidf_text, Length: 9767, dtype: object

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y_class)

print("Mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

Mapping: {'Easy': 0, 'Hard': 1, 'Medium': 2}


In [8]:
y_class.value_counts

<bound method IndexOpsMixin.value_counts of 0          Easy
1          Easy
2        Medium
3          Hard
4          Hard
          ...  
10019      Hard
10020      Hard
10021      Hard
10022      Easy
10023      Easy
Name: difficulty, Length: 9767, dtype: object>

In [9]:
df.columns

Index(['time_limit', 'memory_limit', 'title', 'description', 'input_format',
       'output_format', 'rating', 'tags', 'difficulty', 'combined_text',
       'tokens', 'tokens_clean', 'algo_count', 'tokens_lemma', 'word_count',
       'token_count', 'complexity_terms', 'tfidf_text'],
      dtype='object')

As there are a lot of tags we are using it to understand its relation with difficulty.

In [10]:
print(df['tags'].iloc[0])
print(type(df['tags'].iloc[0]))
import re
def parse_numpy_tag_string(s):
    if not isinstance(s, str):
        return []
    # extract quoted substrings
    return re.findall(r"'([^']+)'", s)

df['tags'] = df['tags'].apply(parse_numpy_tag_string)

['math' 'number theory']
<class 'str'>


Only keeping features essential for training.

In [11]:
to_keep=['tfidf_text','tags','word_count','algo_count','complexity_terms','token_count','time_limit','memory_limit']
X=df[to_keep]


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test = train_test_split(
    X,
    y_score,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded   
)

X_train_final = tfidf.fit_transform(X_train['tfidf_text'])
X_test_final = tfidf.transform(X_test['tfidf_text'])


In [13]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

train_tags = mlb.fit_transform(X_train['tags'])
test_tags  = mlb.transform(X_test['tags'])


Scaling all numerical features 

In [14]:
from sklearn.preprocessing import StandardScaler

numeric_cols = ['word_count','complexity_terms','algo_count','token_count','time_limit','memory_limit']
scaler = StandardScaler()
X_num_train_scaled = scaler.fit_transform(X_train[numeric_cols])
X_num_test_scaled = scaler.transform(X_test[numeric_cols])


Combining all the numerical and text data by horizontally stacking.

In [15]:
from scipy.sparse import hstack,csr_matrix

X_train_final = hstack([X_train_final, X_num_train_scaled,csr_matrix(train_tags)])
X_test_final = hstack([X_test_final, X_num_test_scaled,csr_matrix(test_tags)])

Applying classification models for prediction .

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
lr = LogisticRegression(max_iter=1000,multi_class='multinomial',random_state=42)
lr.fit(X_train_final, y_clf_train)
y_pred_lr=lr.predict(X_test_final)
print(classification_report(y_clf_test,y_pred_lr))
print(confusion_matrix(y_clf_test,y_pred_lr))
accuracy_score(y_clf_test,y_pred_lr)

c:\Users\Akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.69      0.71      0.70       480
           1       0.77      0.79      0.78       846
           2       0.53      0.49      0.51       628

    accuracy                           0.68      1954
   macro avg       0.66      0.66      0.66      1954
weighted avg       0.67      0.68      0.67      1954

[[341  16 123]
 [ 25 671 150]
 [131 189 308]]


0.6755373592630501

In [17]:
from sklearn.svm import LinearSVC

svc_cls = LinearSVC(C=1,class_weight='balanced',random_state=42)
svc_cls.fit(X_train_final, y_clf_train)
y_pred_svc=svc_cls.predict(X_test_final)
print(confusion_matrix(y_clf_test,y_pred_svc))
accuracy_score(y_clf_test,y_pred_svc)


[[345  27 108]
 [ 34 630 182]
 [162 188 278]]


0.6412487205731832

In [18]:
from sklearn.utils.class_weight import compute_sample_weight
import xgboost  as xgb

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_clf_train
)

model_xgb_clf = xgb.XGBClassifier(
    n_estimators=250,      
    max_depth=6,           
    learning_rate=0.1,     
    objective='multi:softprob', 
    random_state=42,
    tree_method='hist'     
)

model_xgb_clf.fit(X_train_final, y_clf_train, sample_weight=sample_weights)
y_pred_xgb_clf = model_xgb_clf.predict(X_test_final)

print("XGBoost Performance:")
print(classification_report(y_clf_test, y_pred_xgb_clf))
print(confusion_matrix(y_clf_test,y_pred_xgb_clf))
print(f"Accuracy: {accuracy_score(y_clf_test, y_pred_xgb_clf)}")


XGBoost Performance:
              precision    recall  f1-score   support

           0       0.67      0.71      0.69       480
           1       0.79      0.77      0.78       846
           2       0.52      0.51      0.51       628

    accuracy                           0.67      1954
   macro avg       0.66      0.66      0.66      1954
weighted avg       0.67      0.67      0.67      1954

[[340  13 127]
 [ 22 654 170]
 [144 166 318]]
Accuracy: 0.6714431934493347


In [19]:
from sklearn.ensemble import RandomForestClassifier
rfc_cls=RandomForestClassifier(n_estimators=400,class_weight='balanced')
rfc_cls.fit(X_train_final, y_clf_train, sample_weight=sample_weights)
y_pred_rfc=rfc_cls.predict(X_test_final)
print(classification_report(y_clf_test, y_pred_rfc))
print(f"Accuracy: {accuracy_score(y_clf_test, y_pred_rfc)}")

              precision    recall  f1-score   support

           0       0.65      0.63      0.64       480
           1       0.73      0.82      0.77       846
           2       0.49      0.43      0.46       628

    accuracy                           0.65      1954
   macro avg       0.63      0.63      0.63      1954
weighted avg       0.64      0.65      0.64      1954

Accuracy: 0.646878198567042


In [20]:
confusion_matrix(y_clf_test, y_pred_rfc)

array([[301,  29, 150],
       [ 24, 690, 132],
       [135, 220, 273]], dtype=int64)

Applying regression models for prediction of rating .

In [26]:
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.utils.class_weight import compute_sample_weight
import xgboost  as xgb


model_xgb_reg = xgb.XGBRegressor(
    n_estimators=250,       # Number of trees
    max_depth=6,            # Depth of trees
    learning_rate=0.1,      
    random_state=42,
    tree_method='hist'      # Faster training for 2025 datasets
)

model_xgb_reg.fit(X_train_final, y_reg_train)

y_pred_xgb_reg = model_xgb_reg.predict(X_test_final)

print("R2 score:")
print(r2_score(y_reg_test,y_pred_xgb_reg))
print("MAE:")

mean_absolute_error(y_reg_test, y_pred_xgb_reg)

R2 score:
0.5819431042294888
MAE:


355.2689625360339

In [27]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(
    n_estimators=50, 
    max_depth=10, 
    n_jobs=2, 
    random_state=42
)

rfr.fit(X_train_final, y_reg_train)
y_pred_rfr=rfr.predict(X_test_final)
print("R2 score")
print(r2_score(y_reg_test, y_pred_rfr))
print("MAE")

mean_absolute_error(y_reg_test, y_pred_rfr)

R2 score
0.5227730001220499
MAE


385.7847424368066

Saving all the preprocessing artifacts .

In [28]:
import pickle

pickle.dump(tfidf, open("../web_ui/tfidf.pkl", "wb"))
pickle.dump(scaler, open("../web_ui/scaler.pkl", "wb"))
pickle.dump(mlb, open("../web_ui/mlb_tags.pkl", "wb"))
pickle.dump(le, open("../web_ui/label_encoder.pkl", "wb"))

pickle.dump(lr, open("../best_models/clf_model.pkl", "wb"))
pickle.dump(model_xgb_reg, open("../best_models/reg_model.pkl", "wb"))
